In [1]:
import numpy as np
from tqdm import tqdm

In [2]:
def link_prediction(TTM_file, TTIE_file, sub_1, sub_2, network_size):
    ad_matrix1 = np.zeros((network_size, network_size))
    ad_matrix2 = np.zeros((network_size, network_size))
    
    fr_1 = open(sub_1)
    for line in fr_1.readlines():
        arr_line = line.strip().split('\t')
        node_1 = int(arr_line[0])
        node_2 = int(arr_line[1])
        ad_matrix1[node_1, node_2] = 1
    
    link_set = []
    fr_2 = open(sub_2)
    for line in fr_2.readlines():
        arr_line = line.strip().split('\t')
        node_1 = int(arr_line[0])
        node_2 = int(arr_line[1])
        ad_matrix2[node_1, node_2] = 1 
        link_set.append([node_1, node_2])
        
    TTM = np.zeros((64, 64))
    fr_3 = open(TTM_file)
    index = 0
    for line in fr_3.readlines():
        
        arr_line = line.strip().split()
        for i in range(64):
            TTM[index, i] = float(arr_line[i])
        index += 1
    
    TTIE = np.zeros((64, 64))
    np.random.seed(2)
    fr_4 = open(TTIE_file)
    index = 0
    for line in fr_4.readlines():
        arr_line = line.strip().split()
        for i in range(64):
            TTIE[index, i] = float(arr_line[i])
        index += 1
    
    AUC = 0
    
    for i in tqdm(range(len(link_set))):
#     for i in range(len(link_set)):
        index = i
        
    
#     for i in tqdm(range(N)):
#         index = np.random.randint(0, len(link_set))
        node_a = link_set[index][0]
        node_b = link_set[index][1]
        score_ab = 0
        
        for node_c in range(network_size):
            node_list = [node_a, node_b, node_c]
            node_list.sort()
            id_1 = int(ad_matrix1[node_list[0],node_list[2]] + ad_matrix1[node_list[1],node_list[2]]*2 + ad_matrix1[node_list[2],node_list[0]]*4 + ad_matrix1[node_list[1],node_list[0]]*8 + ad_matrix1[node_list[2],node_list[1]] *16 + ad_matrix1[node_list[0],node_list[1]]*32)
            if node_a > node_b :
                if node_c<node_b:
                    for x in range(64):
                        dd = 0b100000
                        if (x&dd) == dd:
                            score_ab += TTM[id_1, x] * TTIE[id_1, x]
                if node_c > node_a:
                    for x in range(64):
                        dd = 0b000010
                        if (x&dd) == dd:
                            score_ab += TTM[id_1, x] * TTIE[id_1, x]
                if node_c < node_a and node_c > node_b:
                    for x in range(64):
                        dd = 0b000001
                        if (x&dd) == dd:
                            score_ab += TTM[id_1, x] * TTIE[id_1, x]
            else:
                if node_c < node_a:
                    for x in range(64):
                        dd = 0b001000
                        if (x&dd) == dd:
                            score_ab += TTM[id_1, x]
                if node_c > node_b:
                    for x in range(64):
                        dd = 0b010000
                        if (x&dd) == dd:
                            score_ab += TTM[id_1, x]
                if node_c < node_b and node_c > node_a:
                    for x in range(64):
                        dd = 0b000100
                        if (x&dd) == dd:
                            score_ab += TTM[id_1, x]
        
        score_de = 0
        flag = True
        node_d, node_e = 0, 0
        while(flag):
            node_d = np.random.randint(0, network_size)
            node_e = np.random.randint(0, network_size)
            if node_d != node_e and ad_matrix2[node_d, node_e] == 0:
                flag = False
        for node_f in range(network_size):
            node_list = [node_d, node_e, node_f]
            node_list.sort()
            id_1 = int(ad_matrix1[node_list[0],node_list[2]] + ad_matrix1[node_list[1],node_list[2]]*2 + ad_matrix1[node_list[2],node_list[0]]*4 + ad_matrix1[node_list[1],node_list[0]]*8 + ad_matrix1[node_list[2],node_list[1]] *16 + ad_matrix1[node_list[0],node_list[1]]*32)
            if node_d > node_e:
                if node_f< node_e:
                    for x in range(64):
                        dd = 0b100000
                        if (x&dd) == dd:
                            score_de += TTM[id_1, x] * TTIE[id_1, x]
                if node_f > node_d:
                    for x in range(64):
                        dd = 0b000010
                        if (x&dd) == dd:
                            score_de += TTM[id_1, x] * TTIE[id_1, x]
                if node_f < node_d and node_f > node_e:
                    for x in range(64):
                        dd = 0b000001
                        if (x&dd) == dd:
                            score_de += TTM[id_1, x] * TTIE[id_1, x]
            else:
                if node_f < node_d:
                    for x in range(64):
                        dd = 0b001000
                        if (x&dd) == dd:
                            score_de += TTM[id_1, x] * TTIE[id_1, x]
                if node_f > node_e:
                    for x in range(64):
                        dd = 0b010000
                        if (x&dd) == dd:
                            score_de += TTM[id_1, x] * TTIE[id_1, x]
                if node_f < node_e and node_f > node_d:
                    for x in range(64):
                        dd = 0b000100
                        if (x&dd) == dd:
                            score_de += TTM[id_1, x] * TTIE[id_1, x]
                            
        if score_ab>score_de:
            AUC +=1
        if score_ab == score_de:
            AUC += 0.5
    AUC = AUC / len(link_set)
#     AUC = AUC / N
    return AUC

## 各网络链路预测结果

In [3]:
# enron 

AUC = link_prediction('./predict_TTMs/enron49_50.txt', './TTIE_matrix.txt', './enron_sub/49.txt', './enron_sub/50.txt', 2115)
AUC


100%|██████████| 1451/1451 [02:13<00:00, 10.89it/s]


0.9286698828394211

In [4]:
# facebook

AUC = link_prediction('./predict_TTMs/facebook8_9.txt', './TTIE_matrix.txt', './facebook_sub/8.txt', './facebook_sub/9.txt', 5111)
AUC


100%|██████████| 3177/3177 [11:41<00:00,  4.53it/s]


0.7423670129052565

In [6]:
# col_ms

AUC = link_prediction('./predict_TTMs/col24_25.txt', './TTIE_matrix.txt', './col_ms_sub/24.txt', './col_ms_sub/25.txt', 1899)
AUC


100%|██████████| 195/195 [00:16<00:00, 12.07it/s]


0.8589743589743589

In [7]:
# email-eu

AUC = link_prediction('./predict_TTMs/email69_70.txt', './TTIE_matrix.txt', './email_eu_sub/69.txt', './email_eu_sub/70.txt', 1005)

AUC

100%|██████████| 6221/6221 [04:25<00:00, 23.42it/s]


0.9281466002250442

## Enron 连续链路预测结果

In [8]:
for i in range(40, 50):
    TTM_file = "./predict_TTMS/enron" + str(i) + "_" + str(i+1) + ".txt"
    TTIE_file = './TTIE_matrix.txt'
    sub_1 = './enron_sub/'+ str(i) + '.txt'
    sub_2 = './enron_sub/' + str(i+1) + '.txt'
    AUC = link_prediction(TTM_file, TTIE_file, sub_1, sub_2, 2115)
    print(AUC)
    

  0%|          | 0/4844 [00:00<?, ?it/s]

0.9349056603773584


  0%|          | 0/4002 [00:00<?, ?it/s]

0.9323905862923204


  0%|          | 2/3294 [00:00<04:43, 11.59it/s]

...

0.9156626506024096


  0%|          | 1/1451 [00:00<02:25,  9.96it/s]

0.9307944307944308


100%|██████████| 1451/1451 [02:12<00:00, 10.94it/s]

0.9286698828394211


In [10]:
# artificial network
for i in range(3, 12):
    TTM_file = "./predict_TTMS/art" + str(i) + "_" + str(i+1) + ".txt"
    TTIE_file = './TTIE_matrix.txt'
    sub_1 = './art_sub/'+ str(i) + '.txt'
    sub_2 = './art_sub/' + str(i+1) + '.txt'
    AUC = link_prediction(TTM_file, TTIE_file, sub_1, sub_2, 200)
    print(AUC)

  1%|          | 14/1394 [00:00<00:10, 132.84it/s]

0.9036144578313253


  1%|          | 13/1394 [00:00<00:10, 127.97it/s]

0.8593974175035868


  1%|          | 14/1394 [00:00<00:10, 131.21it/s]

...

0.9296325254104769


  1%|          | 14/1279 [00:00<00:09, 130.65it/s]

0.9093041438623924


100%|██████████| 1279/1279 [00:10<00:00, 120.13it/s]

0.9179046129788898
